# Assistant Skill

### Initialize

In [ ]:
import requests
import os
import re
import json
import openai
from dotenv import load_dotenv
from collections import defaultdict

## Retrieve environment variables
load_dotenv()

openai.api_key = OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

MEALIE_API_KEY = os.getenv("MEALIE_API_KEY")
MEALIE_URL = os.getenv("MEALIE_URL")
MEALIE_LIST_ID = os.getenv("MEALIE_LIST_ID")

NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_DATABASE_ID_TASKS = os.getenv("NOTION_DATABASE_ID_TASKS")
NOTION_TASK_ID_GENERAL = os.getenv("NOTION_TASK_ID_GENERAL")
NOTION_TASK_ID_CHATS = os.getenv("NOTION_TASK_ID_CHATS")

S3_URI_1MIN_SILENCE = os.getenv("S3_URI_1MIN_SILENCE")

## Headers for authentication
OPENAI_HEADERS = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json"
}
OPENROUTER_HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json"
}
MEALIE_HEADERS = {
    "Authorization": f"Bearer {MEALIE_API_KEY}",
    "Content-Type": "application/json"
}
NOTION_HEADERS = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

## API settings for selected model
MODEL_MAIN = "gpt-4o-mini"
# MODEL_MAIN = "google/gemini-2.0-flash-thinking-exp:free"
MODEL_FREE = "gpt-4o-mini"
# MODEL_FREE = "google/gemini-2.0-flash-thinking-exp:free"
MODEL_API_URL = "https://openrouter.ai/api/v1/chat/completions"
MODEL_HEADERS = OPENROUTER_HEADERS

## Parameters
MAX_INPUT_CHARS = 10000
MAX_OUTPUT_TOKENS = 300

## Other
WAIT_INVOCATIONS = ["wait", "hold on", "pause", "hang on", "just a moment", "let me think", "give me a second"]
SAVE_INVOCATIONS = ["save", "save session", "save chat history"]
DELETE_INVOCATIONS = ["please delete persistent attributes"]
RESET_INVOCATIONS = ["please reset session parameters"]
MEMORY_INVOCATIONS = ["please update memory", "update memory", "refresh memory"]

UPDATE_MEMORY_PROMPT = f"""
Given the previous message history between a user and an assistant, extract only the most relevant long-term details that would be helpful to persist across sessions.
If you are provided with existing memory, make adjustments to where appropriate.
Do not include temporary problems, one-time errors, or short-lived goals.
Focus on high-value patterns such as tools being used, user preferences, recurring project themes, workflows being set up, or any strong indicators of personal or technical context.
Avoid duplicating similar facts and aim for clarity and minimalism.
Return the extracted memory in concise, neutral-language bullet points, as if writing an internal assistant profile to improve helpfulness in future conversations.
"""

### Skill Session Attributes

In [ ]:
def check_session_attr(session_attr = {}, persistent_attr = {}, bool_reset = False):
    ## Attempt to copy from persistent attributes
    if not session_attr:
        session_attr.update(persistent_attr)
        _ = session_attr.pop("launch_timestamp", None)
        _ = session_attr.pop("session_page_id", None)
    
    ## Initialize session page
    if "launch_timestamp" not in session_attr:
        session_attr["launch_timestamp"] = datetime.now(TZ).strftime("%Y-%m-%d %H:%M:%S")
    
    if "chat_history" not in session_attr:
        session_attr["chat_history"] = []
    ## TODO: Truncate
    if "memory" not in session_attr:
        session_attr["memory"] = "None."
    
    ## Default models
    if "model_main" not in session_attr or bool_reset:
        session_attr["model_main"] = MODEL_MAIN
    if "model_free" not in session_attr or bool_reset:
        session_attr["model_free"] = MODEL_FREE
    session_attr["model_api_url"] = MODEL_API_URL
    
    ## Default Parameters
    if "max_input_chars" not in session_attr or bool_reset:
        session_attr["max_input_chars"] = MAX_INPUT_CHARS
    if "max_output_tokens" not in session_attr or bool_reset:
        session_attr["max_output_tokens"] = MAX_OUTPUT_TOKENS
    
    return session_attr

In [ ]:
session_attr = check_session_attr({})
session_attr

In [ ]:
def update_session_attr(query, session_attr = check_session_attr({})):
    ## TODO: Update session attributes based on the query
    return session_attr

### General Response

In [ ]:
def general_response(query, session_attr = {}):
    """Generates a general response to a query."""
    try:
        ## Check session attributes
        session_attr = check_session_attr(session_attr)

        messages = [{
            "role": "system", "content": 
            "You are an AI voice assistant with the personality of a dignified, professional, and devoted butler named Alfred. "
            "Assist users by answering questions, providing information, asking clarifying questions, offering suggestions, and acting as a sounding board. "
            "You are designed to engage in natural, real-time conversations. "
            "Be conversationally concise with your responses. "
            "Avoid unnecessary elaboration unless the user requests more details. "
            "If you do not know the answer to a question, admit it honestly and suggest alternative ways the user might find the information.\n\n"
            "Memory of users and conversation for context:\n" + session_attr["memory"]
        }]
        
        total_chars = 0
        for timestamp, question, answer in reversed(session_attr["chat_history"]):
            user_message = f"[{timestamp}] {question}"
            assistant_message = answer
            if (total_chars + len(user_message) + len(assistant_message)) > session_attr["max_input_chars"]:
                break
            messages.append({"role": "user", "content": user_message})
            messages.append({"role": "assistant", "content": assistant_message})
            total_chars += len(user_message) + len(assistant_message)
        
        messages.reverse()
        messages.append({"role": "user", "content": query})
        
        data = {
            "model": session_attr["model_free"],
            "messages": messages,
            "max_tokens": session_attr["max_output_tokens"],
            "temperature": 0
        }
        
        response = requests.post(session_attr["model_api_url"], 
                                 headers=MODEL_HEADERS, data=json.dumps(data))
        response_data = response.json()
        if response.ok:
            return response_data['choices'][0]['message']['content']
        else:
            return f"Error {response.status_code}: {response_data.get('error', {}).get('message', 'Unknown error')}"
    except Exception as e:
        return f"Error generating response: {str(e)}"

In [ ]:
general_response("this is a test")

### Add Item to Mealie List

In [ ]:
## Get all Mealie shopping lists
response = requests.get(f"{MEALIE_URL}/households/shopping/lists", headers=MEALIE_HEADERS)
lists = response.json()["items"]
[{"name": list["name"], "id": list["id"]} for list in lists]

In [ ]:
payload = {
  "shoppingListId": MEALIE_LIST_ID,
  "note": "tomatoes"
}

response = requests.post(f"{MEALIE_URL}/households/shopping/items", headers=MEALIE_HEADERS, data=json.dumps(payload))

if response.status_code == 201:
    print(f"Successfully added: {payload['note']}")
else:
    print(f"Failed to add {payload['note']}: {response.status_code}, {response.text}")

In [ ]:
def add_to_mealie_list(item):
    try:
        ## Create the payload for the API request
        payload = {
            "shoppingListId": MEALIE_LIST_ID,
            "note": item
        }

        ## Send the request to add the item to the shopping list
        response = requests.post(f"{MEALIE_URL}/households/shopping/items", 
                                 headers=MEALIE_HEADERS, data=json.dumps(payload))

        if response.status_code == 201:
            return f"Successfully added {item}"
        else:
            return f"Unsuccessful adding {item}: {response.status_code}, {response.text}"
        
    except Exception as e:
        return f"Error occurred: {str(e)}"

In [ ]:
query = "add 1 cup of sugar"
item = re.sub(r"^add\s+", "", query, flags=re.I).strip()

add_to_mealie_list(item)

### Add Item to Notion Database

In [ ]:
def create_notion_task(title, parent_page_id=NOTION_TASK_ID_GENERAL, content=None):
    try:
        # Create the payload for the API request
        payload = {
            "parent": {"database_id": NOTION_DATABASE_ID_TASKS},
            "properties": {
                "Name": {
                    "title": [
                        {
                            "text": {
                                "content": title
                            }
                        }
                    ]
                },
                "Parent task": {
                    "relation": [
                        {
                            "id": parent_page_id
                        }
                    ]
                }
            }
        }

        # Add optional content as children if provided
        if content:
            payload["children"] = content

        # Send the request to create the page
        response = requests.post("https://api.notion.com/v1/pages", 
                                 headers=NOTION_HEADERS, 
                                 data=json.dumps(payload))

        if response.status_code == 200:
            page_id = response.json().get("id")
            print(f"Page '{title}' created successfully with ID: {page_id}.")
            return page_id
        else:
            print(f"Failed to create page '{title}': {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

In [ ]:
create_notion_task("This is a test")

In [ ]:
def find_or_create_notion_task(title, parent_page_id=NOTION_TASK_ID_GENERAL, content=None):
    try:
        # Search for a page with the same title and parent task
        query_payload = {
            "filter": {
                "and": [
                    {
                        "property": "Name",
                        "title": {
                            "equals": title
                        }
                    },
                    {
                        "property": "Parent task",
                        "relation": {
                            "contains": parent_page_id
                        }
                    }
                ]
            }
        }

        # Send the request to search for the page
        search_response = requests.post(f"https://api.notion.com/v1/databases/{NOTION_DATABASE_ID_TASKS}/query",
                                        headers=NOTION_HEADERS,
                                        data=json.dumps(query_payload))

        if search_response.status_code == 200:
            results = search_response.json().get("results", [])
            if results:
                page_id = results[0]["id"]
                print(f"Page '{title}' already exists with ID: {page_id}.")
                return page_id
            else:
                print(f"No existing page found for title '{title}'. Creating a new one.")
        else:
            print(f"Failed to search for page '{title}': {search_response.status_code}, {search_response.text}")
            return None

        # Create the page if it doesn't exist
        return create_notion_task(title, parent_page_id, content)

    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

In [ ]:
find_or_create_notion_task("This is a test")

### Smart Add to Mealie or Notion

In [ ]:
def smart_add_item(input_value, session_attr = {}):
    try:
        ## Check session attributes
        session_attr = check_session_attr(session_attr)

        ## Define the prompt
        prompt = f"""
        You are a smart assistant. Analyze the following input and split it into individual items. 
        For each item, determine if it is a shopping item or a task/note.
        Respond with a JSON array where each element is a dictionary with "function" and "item" attributes.
        The output format should be a raw JSON array without formatting (such as code block) that can be interpretable with json.loads().
        "function" should be "add_to_mealie_list" for shopping items and "create_notion_task" for tasks and notes.
        Input: "{input_value}"
        """

        ## Call model to analyze the input
        payload = {
            "model": session_attr["model_main"],
            "messages": [{"role": "user", "content": prompt}]
        }
        response = requests.post(session_attr["model_api_url"], 
                                 headers=MODEL_HEADERS, data=json.dumps(payload))

        if response.status_code != 200:
            return f"Error calling {payload['model']}: {response.status_code}, {response.text}"

        ## Parse the response
        response_json = response.json()
        response_json = response_json["choices"][0]["message"]["content"]
        decisions = json.loads(response_json)
        print(decisions)

        mealie_items = []
        notion_items = []

        ## Execute the appropriate function for each decision
        for decision in decisions:
            if decision["function"] == "add_to_mealie_list":
                add_to_mealie_list(decision["item"])
                mealie_items.append(decision["item"])
            elif decision["function"] == "create_notion_task":
                create_notion_task(decision["item"])  # Create regardless of existing task
                notion_items.append(decision["item"])
            print(f"Decision: {decision['function']} - {decision['item']}")

        ## Construct the result string
        mealie_str = f"added {', '.join(mealie_items)} to mealie" if mealie_items else ""
        notion_str = f"added {', '.join(notion_items)} to notion" if notion_items else ""
        result_str = " and ".join(filter(None, [mealie_str, notion_str]))

        return result_str

    except Exception as e:
        return f"Error occurred: {str(e)}"

In [ ]:
query = "add 3 potatoes and mow the lawn, and I need to file taxes and pick up oranges and lemons"
input_value = re.sub(r"^add\s+", "", query, flags=re.I).strip()
        
smart_add_item(input_value)

### Save Session to Notion

In [ ]:
def save_session_to_notion(session_attr):
    """
    Save chat history to a Notion page.
    """
    try:
        session_attr = check_session_attr(session_attr)
        
        chat_history = session_attr["chat_history"]
        chat_history = [entry for entry in chat_history if entry[0] >= session_attr["launch_timestamp"]]
        if not chat_history:
            return "No chat history provided to save."
            
        if "session_page_id" not in session_attr:
            session_attr["session_page_id"] = find_or_create_notion_task(f"Session {session_attr['launch_timestamp']}", parent_page_id=NOTION_TASK_ID_CHATS)
        session_page_id = session_attr["session_page_id"]

        ## Fetch existing content from the Notion page
        response = requests.get(f"https://api.notion.com/v1/blocks/{session_page_id}/children", headers=NOTION_HEADERS)
        if response.status_code != 200:
            return f"Failed to fetch page content: {response.status_code}, {response.text}"
        
        existing_content = response.json()
        existing_texts = set()
        for block in existing_content.get("results", []):
            if block["type"] == "paragraph" and block["paragraph"]["rich_text"]:
                existing_texts.add(block["paragraph"]["rich_text"][0]["text"]["content"])
        
        ## Prepare new chat history to add
        new_blocks = []
        for timestamp, query, speak_output in chat_history:
            try:
                # Create main block for the timestamp
                if timestamp not in existing_texts:  # Avoid duplicates
                    new_blocks.append({
                        "object": "block",
                        "type": "paragraph",
                        "paragraph": {
                            "rich_text": [{"type": "text", "text": {"content": timestamp}}],
                            "children": [
                                {
                                    "object": "block",
                                    "type": "paragraph",
                                    "paragraph": {
                                        "rich_text": [
                                            {
                                                "type": "text",
                                                "text": {
                                                "content": query
                                                },
                                                "annotations": {
                                                "bold": True,
                                                "italic": True
                                                }
                                            }
                                        ]
                                    }
                                },
                                {
                                    "object": "block",
                                    "type": "paragraph",
                                    "paragraph": {
                                        "rich_text": [{"type": "text", "text": {"content": f"{speak_output}"}}]
                                    }
                                }
                            ]
                        }
                    })
            except Exception as e:
                return f"Error processing chat history entry: {str(e)}"
        
        if not new_blocks:
            return f"No new chat history to save."
        
        ## Add new chat history to the Notion page
        payload = {"children": new_blocks}
        response = requests.patch(f"https://api.notion.com/v1/blocks/{session_page_id}/children", 
                                  headers=NOTION_HEADERS, data=json.dumps(payload))
        
        if response.status_code == 200:
            return f"Successfully saved."
        else:
            return f"Error updating page: {response.status_code}, {response.text}"
    
    except Exception as e:
        return f"Error occurred: {str(e)}"

In [ ]:
session_attr = {
			"model_free": "gpt-4o-mini",
			"launch_timestamp": "2025-04-14 23:25:19",
			"model_main": "gpt-4o-mini",
			"model_api_url": "https://openrouter.ai/api/v1/chat/completions",
			"max_input_chars": 10000,
			"chat_history": [
				[
					"2025-04-14 21:59:50",
					"how are you",
					"I'm doing well, thanks! How about you?"
				],
				[
					"2025-04-14 23:08:52",
					"what's on my shopping list",
					"25 items. 12 Produce: limes wedges, for serving, cilantro or green onions, chopped, 8 roma tomatoes large, or 0.25x lbs cherry or a combination, 1 red bell pepper medium, 2 rosemary springs, fresh, basil optional, ³/₄ lb tomatillo quartered, husks discarded, about 5.33x large tomatillos, ²/₃ lb poblano pepper roughly chopped, seeds and stems discarded, about 3x peppers, 6 oz Anaheim peppers roughly chopped, seeds and stems discarded, about 0.33x peppers, 2 jalapeno peppers or serrano, roughly chopped, stems discarded, 8 oz white onion medium, roughly chopped, ¹/₂ cup cilantro leaves and fine stems, loosely packed, plus more for garnish. 3 Canned: 4 oz canned diced green chiles, evaporated milk Costco Business Center, tomato paste tube, Walmart. 2 Dairy: 4 cups heavy whipping cream, 1 ¹/₂ cups Monterey Jack cheese shredded, or Mexican blend cheese. 1 Bakery: 8 flour tortillas about 8-10 inches in diameter. 1 Meat: 9 oz pork chorizo specifically Cacique. 5 Seasoning: guacamole, Japanese red pickled ginger 99 Ranch, Tokyo Central, sesame oil 99 Ranch, 4 lbs sugar Walmart, 1 cup salsa verde jarred. 1 No Label: gum from Walmart or Costco Business Center"
				],
				[
					"2025-04-14 23:09:14",
					"what produce do I need",
					"You need the following produce: \n\n- Limes (for serving)\n- Cilantro or green onions (chopped)\n- 8 large Roma tomatoes or about 0.25 lbs of cherry tomatoes\n- 1 medium red bell pepper\n- 2 sprigs of fresh rosemary\n- ¾ lb tomatillos (quartered, husks discarded)\n- ⅔ lb poblano peppers (roughly chopped, seeds and stems discarded)\n- 6 oz Anaheim peppers (roughly chopped, seeds and stems discarded)\n- 2 jalapeño or serrano peppers (roughly chopped, stems discarded)\n- 8 oz medium white onion (roughly chopped)\n- ½ cup cilantro leaves and fine stems (loosely packed, plus more for garnish)\n\nNeed anything else?"
				],
				[
					"2025-04-14 23:09:38",
					"give me just the items",
					"Sure! Here are the items on your shopping list:\n\n**Produce:**\n- Limes (for serving)\n- Cilantro or green onions (chopped)\n- 8 large Roma tomatoes or 0.25 lbs cherry tomatoes\n- 1 medium red bell pepper\n- 2 sprigs fresh rosemary\n- ¾ lb tomatillos (quartered, husks discarded)\n- ⅔ lb poblano peppers (roughly chopped, seeds and stems discarded)\n- 6 oz Anaheim peppers (roughly chopped, seeds and stems discarded)\n- 2 jalapeño or serrano peppers (roughly chopped, stems discarded)\n- 8 oz medium white onion (roughly chopped)\n- ½ cup cilantro leaves and fine stems (loosely packed, plus more for garnish)\n\n**Canned:**\n- 4 oz canned diced green chiles\n- Evaporated milk\n- Tomato paste (tube)\n\n**Dairy:**\n- 4 cups heavy whipping cream\n- 1 ½ cups shredded Monterey Jack cheese or Mexican blend cheese\n\n**Bakery:**\n- 8 flour tortillas (about 8-10 inches in diameter)\n\n**Meat:**\n- 9 oz pork chorizo (specifically Cacique)\n\n**Seasoning:**\n- Guacamole\n- Japanese red pickled ginger\n- Sesame oil\n- 4 lbs sugar\n- 1 cup jarred salsa verde\n\n**No Label:**\n- Gum\n\nLet me"
				],
				[
					"2025-04-14 23:09:58",
					"give me produce items but just the items without the descriptions or amounts",
					"Sure! Here are the produce items:\n\n- Limes\n- Cilantro or green onions\n- Roma tomatoes or cherry tomatoes\n- Red bell pepper\n- Fresh rosemary\n- Tomatillos\n- Poblano peppers\n- Anaheim peppers\n- Jalapeño or serrano peppers\n- White onion\n- Cilantro leaves and fine stems"
				],
				[
					"2025-04-14 23:12:04",
					"what's the weather in Sunnyvale right now",
					"I don't have real-time weather data. You can check a weather website or app for the current conditions in Sunnyvale. Would you like tips on where to find that information?"
				],
				[
					"2025-04-14 23:12:22",
					"summarize our conversation",
					"You asked about the current weather in Sunnyvale, requested a list of produce items without descriptions, and then asked for your complete shopping list. Finally, you checked in on how I was doing. Would you like to discuss anything else?"
				],
				[
					"2025-04-14 23:25:26",
					"summarize our conversation",
					"You asked how I was doing, then inquired about your shopping list and specifically the produce items. You requested the produce items without descriptions or amounts. Finally, you asked about the current weather in Sunnyvale."
				]
			],
			"max_output_tokens": 300
		}
save_session_to_notion(page_id=None, session_attr=session_attr)

### Get Shopping List

In [ ]:
def get_shopping_list():
    ## Retrieve items in shopping list
    response = requests.get(f"{MEALIE_URL}/households/shopping/items?orderBy=checked&orderDirection=asc&page=1&perPage=100&checked=false", 
                        headers=MEALIE_HEADERS)

    items = [{"item": item["display"], "label": item["label"]["name"] if "label" in item and item["label"] else "No Label"} 
            for item in response.json()["items"] if item["checked"] == False]

    ## Group items by their labels
    grouped_items = defaultdict(list)
    for entry in items:
        grouped_items[entry['label']].append("[] " + entry['item'])

    ## Format the string
    total_items = sum(len(items) for items in grouped_items.values())
    items_string = f"{total_items} items.\n" + "\n".join(
        f"{len(items)} {label}:\n" + "\n".join(items) for label, items in grouped_items.items()
    )
    return items_string

In [ ]:
print(get_shopping_list())